# Subset tissue data on the regulatory genes only

In [16]:
library(Matrix)
suppressPackageStartupMessages(library(Seurat))
suppressPackageStartupMessages(library(tidyverse))
suppressPackageStartupMessages(library(viridis))
library(glmGamPoi)
library(RColorBrewer)
library(data.table)
options(future.globals.maxSize = 10000 * 1024^2)
future.seed=TRUE


In [17]:
setwd("/share/crsp/lab/seyedam/share/enc4_mouse/snrna/")

In [18]:
genes = read.csv("ref/new_gene_display_table.csv")

In [19]:
subset_rna = function(seurat_obj_path){
    obj = readRDS(seurat_obj_path)
    rna_counts = obj@assays$RNA@counts
    rna_counts = rna_counts[rownames(rna_counts) %in% genes$gene_name,]
    new_obj = CreateSeuratObject(counts = rna_counts, min.cells = 0, min.features = 0)
    new_obj@meta.data$cellID = obj@meta.data$cellID
    new_obj@meta.data$file_accession = obj@meta.data$file_accession
    new_obj@meta.data$library_accession = obj@meta.data$library_accession
    new_obj@meta.data$experiment_accession = obj@meta.data$experiment_accession
    
    new_obj@meta.data$nCount_RNA = obj@meta.data$nCount_RNA
    new_obj@meta.data$nFeature_RNA = obj@meta.data$nFeature_RNA
    new_obj@meta.data$nCount_SCT = obj@meta.data$nCount_SCT
    new_obj@meta.data$nFeature_SCT = obj@meta.data$nFeature_SCT
    
    new_obj@meta.data$technology = obj@meta.data$technology
    new_obj@meta.data$species = obj@meta.data$species
    new_obj@meta.data$tissue = obj@meta.data$tissue
    new_obj@meta.data$sex = obj@meta.data$sex
    new_obj@meta.data$timepoint = obj@meta.data$timepoint
    new_obj@meta.data$rep = obj@meta.data$rep
    new_obj@meta.data$sample = obj@meta.data$sample
    new_obj@meta.data$depth1 = obj@meta.data$depth1
    new_obj@meta.data$depth2 = obj@meta.data$depth2
    new_obj@meta.data$experiment = obj@meta.data$experiment
    new_obj@meta.data$experiment_batch = obj@meta.data$experiment_batch
    new_obj@meta.data$integration_batch = obj@meta.data$integration_batch
    new_obj@meta.data$run_number = obj@meta.data$run_number
    
    new_obj@meta.data$doublet_scores = obj@meta.data$doublet_scores
    new_obj@meta.data$doublets = obj@meta.data$doublets
    new_obj@meta.data$lower_nCount_RNA = obj@meta.data$lower_nCount_RNA
    new_obj@meta.data$upper_nCount_RNA = obj@meta.data$upper_nCount_RNA
    new_obj@meta.data$lower_nFeature_RNA = obj@meta.data$lower_nFeature_RNA
    new_obj@meta.data$upper_doublet_scores = obj@meta.data$upper_doublet_scores
    new_obj@meta.data$upper_percent.mt = obj@meta.data$upper_percent.mt
    new_obj@meta.data$percent.mt = obj@meta.data$percent.mt
    new_obj@meta.data$percent.ribo = obj@meta.data$percent.ribo

    new_obj@meta.data$S.Score = obj@meta.data$S.Score    
    new_obj@meta.data$G2M.Score = obj@meta.data$G2M.Score  
    new_obj@meta.data$Phase = obj@meta.data$Phase

    new_obj@meta.data$seurat_clusters = obj@meta.data$seurat_clusters
    new_obj@meta.data$subtypes = obj@meta.data$subtypes
    new_obj@meta.data$celltypes = obj@meta.data$celltypes
    new_obj@meta.data$gen_celltype = obj@meta.data$gen_celltype    
    
    out = new_obj
}

## Adrenal

In [20]:
adr  = subset_rna("seurat/adrenal_Parse_10x_integrated.rds")
adr$c2c12_cluster = NA

## Heart

In [21]:
hrt  = subset_rna("seurat/heart_Parse_10x_integrated.rds")
hrt$c2c12_cluster = NA

## Cortex

In [22]:
ctx  = subset_rna("seurat/cortex_Parse_10x_integrated.rds")
ctx$c2c12_cluster = NA

## Hippocampus

In [23]:
hc  = subset_rna("seurat/hippocampus_Parse_10x_integrated.rds")
hc$c2c12_cluster = NA

## Gastroc

In [24]:
gas  = subset_rna("seurat/gastrocnemius_Parse_10x_integrated.rds")
gas$c2c12_cluster = NA

In [25]:
table(gas$celltypes)


   Adipocytes   Endothelial           FAP           MTJ   Macrophages 
          334          5074         11061          1616          1182 
          NMJ     Perinatal     Satellite Schwann_cells Smooth_muscle 
          972          5603          1505           552          2275 
    Tenocytes         Type1        Type2A        Type2B        Type2X 
         3067          1492          2775         30435          4449 

## C2C12

In [26]:
c2  = subset_rna("seurat/c2c12_Parse_integrated.rds")
c2_obj = readRDS("seurat/c2c12_Parse_integrated.rds")
c2$c2c12_cluster = c2_obj$c2c12_cluster

# Merge and save

In [27]:
all_tissue_c2c12 = merge(adr,c(hrt,ctx,hc,gas,c2))

In [28]:
table(all_tissue_c2c12$tissue)


      Adrenal         C2C12        Cortex Gastrocnemius         Heart 
        79209         22234         98867         72392         93522 
  Hippocampus 
        76386 

In [29]:
all_tissue_c2c12

An object of class Seurat 
2701 features across 442610 samples within 1 assay 
Active assay: RNA (2701 features, 0 variable features)

In [30]:
# SAVE
saveRDS(all_tissue_c2c12,file="seurat/all_tissues_Parse_10x_TFs_mirhgs_chromreg.rds")
write.csv(all_tissue_c2c12@meta.data,file="seurat/all_tissues_Parse_10x_TFs_mirhgs_chromreg_metadata.csv")


## Save Parse only

In [31]:
all_tissue_c2c12_parse = subset(all_tissue_c2c12, subset = technology == "Parse")
all_tissue_c2c12_parse
table(all_tissue_c2c12_parse$tissue)
saveRDS(all_tissue_c2c12_parse,file="seurat/all_tissues_Parse_TFs_mirhgs_chromreg.rds")
write.csv(all_tissue_c2c12_parse@meta.data,file="seurat/all_tissues_Parse_TFs_mirhgs_chromreg_metadata.csv")


An object of class Seurat 
2701 features across 217296 samples within 1 assay 
Active assay: RNA (2701 features, 0 variable features)


      Adrenal         C2C12        Cortex Gastrocnemius         Heart 
        44213         22234         44747         30896         32593 
  Hippocampus 
        42613 

## Save 10x only

In [32]:
all_tissue_c2c12_10x = subset(all_tissue_c2c12, subset = technology == "10x")
all_tissue_c2c12_10x
table(all_tissue_c2c12_10x$tissue)
saveRDS(all_tissue_c2c12_10x,file="seurat/all_tissues_10x_TFs_mirhgs_chromreg.rds")
write.csv(all_tissue_c2c12_10x@meta.data,file="seurat/all_tissues_10x_TFs_mirhgs_chromreg_metadata.csv")


An object of class Seurat 
2701 features across 225314 samples within 1 assay 
Active assay: RNA (2701 features, 0 variable features)


      Adrenal        Cortex Gastrocnemius         Heart   Hippocampus 
        34996         54120         41496         60929         33773 